In [1]:
from tqdm import tqdm
# Import the data sets
import os
import sys
import pandas as pd
import numpy as np
base_path = r"C:\Users\KAI\Coding\ThinkOnward_challenge\thinkOnward_TSClassification"
data_path = r"\data\building-instinct-starter-notebook\Starter notebook"
preprocessing_path = r"\kai\preprocessing"
sys.path.append(base_path+data_path)
sys.path.append(base_path+"\kai")
sys.path.append(base_path+preprocessing_path)
from preprocessing import Preprocessor

# df_features_dict = {}

# for s in ["monthly", "weekly", "daily"]:
#     # df_features = pd.read_parquet(base_path + f'/preprocessed_data/{s}_data.parquet', engine='pyarrow')
#     folder_path = os.path.join(os.getcwd(),'building-instinct-train-data')# folder path for the train dataset
#     df_features = Preprocessor.calculate_average_energy_consumption(folder_path, type="daily")
#     # df_features.sort_index(inplace=True)
#     # df_features_dict[s] = df_features

# # Full Dataset
# df_features_full = pd.read_parquet(base_path + '/preprocessed_data/standard_data.parquet', engine='pyarrow')
# df_features_full.sort_index(inplace=True)
# df_features_dict['full'] = df_features_full

# # Labels
# load_filepath_labels = os.path.join(base_path + data_path,'building-instinct-train-label', 'train_label.parquet')#path to the train label file
# df_targets = pd.read_parquet(load_filepath_labels, engine='pyarrow')
# y = df_targets["building_stock_type"].map({"residential": 0, "commercial": 1})

In [2]:
folder_path = os.path.join(base_path + data_path,'building-instinct-train-data')
for file_name in tqdm(os.listdir(folder_path)[:1]):
    if file_name.endswith(".parquet"):
        # Extract the bldg_id from the file name
        bldg_id = int(file_name.split('.')[0])

        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)

        # Read the original parquet file
        df = pd.read_parquet(file_path)

        # Convert 'timestamp' column to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        print(df["in.state"].unique())
        s = df["in.state"].unique()[0]
        print(s)

100%|██████████| 1/1 [00:00<00:00, 19.28it/s]

['KY']
KY


In [12]:
def calculate_average_energy_consumption(folder_path, season_months_dict=None, type='daily', with_regional=False):
    """
    Process multiple parquet files in a folder, calculate average energy consumption,
    and return a pandas DataFrame with each row corresponding to one file in the folder.

    Parameters:
    - folder_path (str): Path to the folder containing parquet files.
    - season_months_dict (dict): A dictionary where keys are season names (strings) and values are lists
    of corresponding month numbers. For example, {'cold': [1, 2, 12], 'hot': [6, 7, 8], 'mild': [3, 4, 5, 9, 10, 11]}.

    Returns:
    - df_ave (pd.DataFrame): A pandas DataFrame with each row corresponding to one file in the folder (i.e. one building).
    The columns are multi-layer with the first layer being the day/week/month/season and the second layer the hour of the day 
    Index ('bldg_id') contains building IDs. Column values are average hourly electricity energy consumption
    """
    # Initialize an empty list to store individual DataFrames for each file
    result_dfs = []
    if with_regional:
        locations = {
            "WI": {"latitude": 44.500000, "longitude": -89.500000},  # Wisconsin
            "WV": {"latitude": 39.000000, "longitude": -80.500000},  # West Virginia
            "VT": {"latitude": 44.000000, "longitude": -72.699997},  # Vermont
            "TX": {"latitude": 31.000000, "longitude": -100.000000}, # Texas
            "SD": {"latitude": 44.500000, "longitude": -100.000000}, # South Dakota
            "RI": {"latitude": 41.742325, "longitude": -71.742332},  # Rhode Island
            "OR": {"latitude": 44.000000, "longitude": -120.500000}, # Oregon
            "NY": {"latitude": 43.000000, "longitude": -75.000000},  # New York
            "NH": {"latitude": 44.000000, "longitude": -71.500000},  # New Hampshire
            "NE": {"latitude": 41.500000, "longitude": -100.000000}, # Nebraska
            "KS": {"latitude": 38.500000, "longitude": -98.000000},  # Kansas
            "MS": {"latitude": 33.000000, "longitude": -90.000000},  # Mississippi
            "IL": {"latitude": 40.000000, "longitude": -89.000000},  # Illinois
            "DE": {"latitude": 39.000000, "longitude": -75.500000},  # Delaware
            "CT": {"latitude": 41.599998, "longitude": -72.699997},  # Connecticut
            "AR": {"latitude": 34.799999, "longitude": -92.199997},  # Arkansas
            "IN": {"latitude": 40.273502, "longitude": -86.126976},  # Indiana
            "MO": {"latitude": 38.573936, "longitude": -92.603760},  # Missouri
            "FL": {"latitude": 27.994402, "longitude": -81.760254},  # Florida
            "NV": {"latitude": 39.876019, "longitude": -117.224121}, # Nevada
            "ME": {"latitude": 45.367584, "longitude": -68.972168},  # Maine
            "MI": {"latitude": 44.182205, "longitude": -84.506836},  # Michigan
            "GA": {"latitude": 33.247875, "longitude": -83.441162},  # Georgia
            "HI": {"latitude": 19.741755, "longitude": -155.844437}, # Hawaii
            "AK": {"latitude": 66.160507, "longitude": -153.369141}, # Alaska
            "TN": {"latitude": 35.860119, "longitude": -86.660156},  # Tennessee
            "VA": {"latitude": 37.926868, "longitude": -78.024902},  # Virginia
            "NJ": {"latitude": 39.833851, "longitude": -74.871826},  # New Jersey
            "KY": {"latitude": 37.839333, "longitude": -84.270020},  # Kentucky
            "ND": {"latitude": 47.650589, "longitude": -100.437012}, # North Dakota
            "MN": {"latitude": 46.392410, "longitude": -94.636230},  # Minnesota
            "OK": {"latitude": 36.084621, "longitude": -96.921387},  # Oklahoma
            "MT": {"latitude": 46.965260, "longitude": -109.533691}, # Montana
            "WA": {"latitude": 47.751076, "longitude": -120.740135}, # Washington
            "UT": {"latitude": 39.419220, "longitude": -111.950684}, # Utah
            "CO": {"latitude": 39.113014, "longitude": -105.358887}, # Colorado
            "OH": {"latitude": 40.367474, "longitude": -82.996216},  # Ohio
            "AL": {"latitude": 32.318230, "longitude": -86.902298},  # Alabama
            "IA": {"latitude": 42.032974, "longitude": -93.581543},  # Iowa
            "NM": {"latitude": 34.307144, "longitude": -106.018066}, # New Mexico
            "SC": {"latitude": 33.836082, "longitude": -81.163727},  # South Carolina
            "PA": {"latitude": 41.203323, "longitude": -77.194527},  # Pennsylvania
            "AZ": {"latitude": 34.048927, "longitude": -111.093735}, # Arizona
            "MD": {"latitude": 39.045753, "longitude": -76.641273},  # Maryland
            "MA": {"latitude": 42.407211, "longitude": -71.382439},  # Massachusetts
            "CA": {"latitude": 36.778259, "longitude": -119.417931}, # California
            "ID": {"latitude": 44.068203, "longitude": -114.742043}, # Idaho
            "WY": {"latitude": 43.075970, "longitude": -107.290283}, # Wyoming
            "NC": {"latitude": 35.782169, "longitude": -80.793457},  # North Carolina
            "LA": {"latitude": 30.391830, "longitude": -92.329102},  # Louisiana
            "DC": {"latitude": 38.907200, "longitude": -77.036900},  # Washington, D.C.
        }

    # Iterate through all files in the folder_path
    for file_name in tqdm(os.listdir(folder_path)):
        if file_name.endswith(".parquet"):
            # Extract the bldg_id from the file name
            bldg_id = int(file_name.split('.')[0])

            # Construct the full file path
            file_path = os.path.join(folder_path, file_name)

            # Read the original parquet file
            df = pd.read_parquet(file_path)

            # Convert 'timestamp' column to datetime
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            df['hour'] = df['timestamp'].dt.hour
            if with_regional:
                state = df["in.state"].unique()[0]
                latitude = locations[state]["latitude"]
                longitude = locations[state]["longitude"]
            if type == 'daily':# -> goes from Input: 365 * 24 * 4 = 35,040 columns to 365 * 24 = 8,760 values per building
                df['day_of_year'] = df['timestamp'].dt.day_of_year
                df['hourly_energy_consumption'] = df.groupby(['day_of_year', 'hour'])['out.electricity.total.energy_consumption'].transform('mean')
                result_df = df.pivot_table(values='hourly_energy_consumption', index='bldg_id', columns=['day_of_year', 'hour'])
            
            elif type == 'weekly':# -> goes from Input: 365 * 24 * 4 = 35,040 columns to 52 * 24 = 1,248 values per building
                df['week'] = df['timestamp'].dt.isocalendar().week
                df['weekly_energy_consumption'] = df.groupby(['week', 'hour'])['out.electricity.total.energy_consumption'].transform('mean')
                result_df = df.pivot_table(values='weekly_energy_consumption', index='bldg_id', columns=['week', 'hour'])

            elif type == 'monthly':# -> goes from Input: 365 * 24 * 4 = 35,040 columns to 12 * 24 = 288 values per building
                df['month'] = df['timestamp'].dt.month
                df['monthly_energy_consumption'] = df.groupby(['month', 'hour'])['out.electricity.total.energy_consumption'].transform('mean')
                result_df = df.pivot_table(values='monthly_energy_consumption', index='bldg_id', columns=['month', 'hour'])

            elif type == 'seasonal': # originally provided prerpocessing method -> goes from Input: 365 * 24 * 4 = 35,040 columns to 365 * (12/s)  = ... values per building
                df['month'] = df['timestamp'].dt.month
                # Create a mapping from month to the corresponding season
                month_to_season = {month: season for season, months_list in season_months_dict.items() for month in months_list}

                # Assign a season to each row based on the month
                df['season'] = df['month'].map(month_to_season)

                # Calculate hourly average energy consumption for each row
                df['hourly_avg_energy_consumption'] = 4 * df.groupby(['season', 'hour'])['out.electricity.total.energy_consumption'].transform('mean')

                # Pivot the dataframe to create the desired output format
                result_df = df.pivot_table(values='hourly_avg_energy_consumption', index='bldg_id', columns=['season', 'hour'])

                # Reset the column names
                result_df.columns = pd.MultiIndex.from_tuples([(season, hour+1) for season, months_list in season_months_dict.items() for hour in range(24)])
            else:
                raise ValueError('Invalid type. Please select from hourly, weekly, or monthly.')

            # Add 'bldg_id' index with values corresponding to the names of the parquet files
            result_df['bldg_id'] = bldg_id
            if with_regional:
                result_df["latitude"] = latitude
                result_df["longitude"] = longitude
            result_df.set_index('bldg_id', inplace=True)

            # Append the result_df to the list
            result_dfs.append(result_df)

    # Concatenate all individual DataFrames into a single DataFrame
    df_ave = pd.concat(result_dfs, ignore_index=False)

    return df_ave

In [13]:
# folder_path = os.path.join(base_path + data_path,'building-instinct-train-data')
# df_features = calculate_average_energy_consumption(folder_path, type="monthly", with_regional=True)
# df_features.sort_index(inplace=True)
# print(df_features)


for s in ["monthly", "weekly", "daily"]:
    save_path = os.path.join(base_path + f'/preprocessed_data/with_regional/{s}_data.parquet')
    # df_features = pd.read_parquet(base_path + f'/preprocessed_data/{s}_data.parquet', engine='pyarrow')
    folder_path = os.path.join(base_path + data_path,'building-instinct-train-data')
    df_features = calculate_average_energy_consumption(folder_path, type=s, with_regional=True)
    df_features.sort_index(inplace=True)
    df_features.to_parquet(save_path, engine='pyarrow')

df_features_full = pd.read_parquet(base_path + '/preprocessed_data/standard_data.parquet', engine='pyarrow')
df_features_full.sort_index(inplace=True)
df_features_full["latitude"] = df_features["latitude"]
df_features_full["longitude"] = df_features["longitude"]
save_path = os.path.join(base_path + f'/preprocessed_data/with_regional/standard_data.parquet')

100%|██████████| 7200/7200 [04:53<00:00, 24.53it/s]
c:\Users\KAI\Coding\ThinkOnward_challenge\thinkOnward_TSClassification\venv\lib\site-packages\pandas\io\parquet.py:159: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)
100%|██████████| 7200/7200 [05:53<00:00, 20.36it/s]
c:\Users\KAI\Coding\ThinkOnward_challenge\thinkOnward_TSClassification\venv\lib\site-packages\pandas\io\parquet.py:159: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)
100%|██████████| 7200/7200 [07:55<00:00, 15.14it/s]
c:\Users\KAI\Coding\ThinkOnward_challenge\thinkOnward_TSClassification\venv\lib\site-packages\pandas\io\parquet.py:159: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip cor

In [15]:
df_features_full.to_parquet(save_path, engine='pyarrow')

c:\Users\KAI\Coding\ThinkOnward_challenge\thinkOnward_TSClassification\venv\lib\site-packages\pandas\io\parquet.py:159: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


In [9]:
print(df_features.state.value_counts())

state
CA    792
TX    551
FL    507
NY    431
PA    313
IL    297
OH    295
NC    238
MI    222
GA    221
VA    202
NJ    186
IN    175
MO    169
TN    166
WA    153
MA    146
CO    143
WI    140
MD    138
AL    126
SC    123
LA    113
MN    112
AZ    108
OK     89
KY     87
OR     86
MS     79
CT     72
AR     69
NV     65
IA     61
UT     56
NM     53
KS     49
NE     44
ME     41
ID     41
WV     37
NH     32
MT     30
RI     28
DE     27
DC     18
WY     18
ND     14
AK     11
SD      9
HI      9
VT      8
Name: count, dtype: int64
